# Data Science Workflow

## 1. Define the Problem



### This problem definition
1. Use Bigmart Sales data to predict the sales of other similar products and stores to the training set.
2. Solving this problem helps with financial projects for stores along with stocking information
3. Solving the problem manually would be to specify all the attributes of each item type and then look for patterns to be able to make predictions

### 2. Data Collection

In [1]:
# Import our favorite data science packages
import os
import numpy as np
import pandas as pd
from scipy import stats

# Import the libraries needed to import data from UCI into pandas
import re
import requests
import io


# We also will need a list of plotting libraries 
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 100

# Obtain the training set
url_name = "https://raw.githubusercontent.com/3Blades/notebook-templates/master/python/Datasets/lp_train.csv"
request=requests.get(url_name).content
df_train=pd.read_csv(io.StringIO(request.decode('utf-8')))
df_train.head()

# Obtain the test set
url_name = "https://raw.githubusercontent.com/3Blades/notebook-templates/master/python/Datasets/lp_test.csv"
request=requests.get(url_name).content
df_test=pd.read_csv(io.StringIO(request.decode('utf-8')))
df_test.head()

# putting the data all together to clean it
data = pd.concat([df_train, df_test], ignore_index=True)

## 2. Prepare Data
1. Data Selection. Availability, what is missing, what can be removed.
2. Data Preprocessing. Organize selected data by formatting, cleaning and sampling.
3. Data Transformation. Feature engineering using scaling, attribute decomposition and attribute aggregation.
4. Data visualizations such as with histograms.


In [2]:
print(data.head())

   ApplicantIncome  CoapplicantIncome  Credit_History Dependents  \
0             5849                0.0             1.0          0   
1             4583             1508.0             1.0          1   
2             3000                0.0             1.0          0   
3             2583             2358.0             1.0          0   
4             6000                0.0             1.0          0   

      Education Gender  LoanAmount  Loan_Amount_Term   Loan_ID Loan_Status  \
0      Graduate   Male         NaN             360.0  LP001002           Y   
1      Graduate   Male       128.0             360.0  LP001003           N   
2      Graduate   Male        66.0             360.0  LP001005           Y   
3  Not Graduate   Male       120.0             360.0  LP001006           Y   
4      Graduate   Male       141.0             360.0  LP001008           Y   

  Married Property_Area Self_Employed  
0      No         Urban            No  
1     Yes         Rural            No  
2 

In [ ]:
# Understanding what data issues we have

# This functions allows me to see 1st level of columns that need to be handled 
def dataCleaningAnalysis(df):
    # columns with out numbers in them
    all_cols = df.columns
    num_cols = df._get_numeric_data().columns
    nonum_cols = list(set(all_cols) - set(num_cols))

    # columns and rows with nans in them
    nancols = df.columns[pd.isnull(df).any()].tolist()

    # columns with 0 in them or other numbers that should not be there
    zerocols = df.columns[pd.notnull(df[df == 0]).any()].tolist()
    return(nonum_cols, nancols, zerocols)

nonum, nan, zero = dataCleaningAnalysis(data)
print("Columns to look at and possible fix\nNo num: {}\nNan: {}\nZero: {}".format(nonum,nan,zero))

# shows how many unique items for each column
data.apply(lambda x: len(x.unique()))

In [ ]:
# Fis the Item_Weight Column
#Determine the average weight per item:
item_avg_weight = data.pivot_table(values='Item_Weight', index='Item_Identifier')

#Get a boolean variable specifying missing Item_Weight values
miss_bool = data['Item_Weight'].isnull() 

#Impute data and check #missing values before and after imputation to confirm
data.loc[miss_bool,'Item_Weight'] = data.loc[miss_bool,'Item_Identifier'].apply(lambda x: item_avg_weight[x])  

print(sum(data['Item_Weight'].isnull()))

# Make the last 8 weight equal to average weight of all items
mn = data['Item_Weight'].mean()
data.loc[data['Item_Weight'].isnull(),'Item_Weight'] = mn
print(sum(data['Item_Weight'].isnull()))


In [ ]:
######### Fix Visibility being 0 with average visibility of each item category
#Determine average visibility of a product
visibility_avg = data.pivot_table(values='Item_Visibility', index='Item_Identifier')
## Find 0 visibility and replace with mean visibility of that product:
miss_bool = (data['Item_Visibility'] == 0)
data.loc[miss_bool,'Item_Visibility'] = data.loc[miss_bool,'Item_Identifier'].apply(lambda x: visibility_avg[x])
print(sum((data['Item_Visibility'] == 0)))

In [ ]:
# Change outlet established years to number of years in business
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

In [ ]:
#### Crete Broad Category
#Get the first two characters of ID:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()


In [ ]:
# Clean up Item_Fat_Content column
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat', 'reg':'Regular', 'low fat':'Low Fat'})
data['Item_Fat_Content'].value_counts()

In [ ]:
####### Add another value to non consumable items
#Mark non-consumables as separate category in low_fat:
data.loc[data['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
data['Item_Fat_Content'].value_counts()


In [ ]:
######### Fix Missing Data - Outlet Size using Store Type
# For missing outlet size data figure out the mode of the outlet types and imput size based on that mode
def modes(df, key_cols, value_col, count_col):
    return df.groupby(key_cols + [value_col]).size() \
             .to_frame(count_col).reset_index() \
             .groupby(key_cols + [count_col])[value_col].unique() \
             .to_frame().reset_index() \
             .sort_values(count_col, ascending=False) \
             .drop_duplicates(subset=key_cols)

print(modes(data,['Outlet_Type'],'Outlet_Size','count'),"\n\n")
######### Fix Missing Data - Outlet Size using Store Type
type_mode = {"Supermarket Type1":"Small","Supermarket Type":"Medium","Supermarket Type2":"Medium", "Grocery Store":"Small"}
miss_bool = data['Outlet_Size'].isnull() 
data.loc[miss_bool,'Outlet_Size'] = data.loc[miss_bool,'Outlet_Type'].apply(lambda x: type_mode[x])
print(sum(data['Outlet_Size'].isnull() ))


In [ ]:
# Encode text data into numbers and take the spaces out of the column names
# Fix Outlet_Size
mapto = {"Small":1,"Medium":2, "High":3}
data['Outlet_Size'] = data['Outlet_Size'].map(mapto)

#One Hot Coding:
data['Outlet'] = data['Outlet_Identifier']
data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Type','Item_Type_Combined','Outlet'])

data.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)


In [ ]:
#Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

#Drop unnecessary columns:
test = test.drop(['source','Item_Type'],axis=1)
train = train.drop(['source','Item_Type'],axis=1)


In [ ]:
# This is the final definition to solving the problem
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
predictors = [x for x in train.columns if x not in [target]+IDcol]

### Prepare Your Data

## 3. Spot Check Algorithms
1. Test harness with default values.
2. Run family of algorithms across all the transformed and scaled versions of dataset.
3. View comparisons with box plots.


In [ ]:
# this uses the mean as a way of testing setting a base level for your algorithmns
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

predict =[]
mean_sales = train['Item_Outlet_Sales'].mean()
predict =  test[target].copy()
predict[:] = mean_sales
rmsbase = sqrt(mean_squared_error(test[target],predict))
print("Base", rmsbase)


### Spot Check Your Data

## 4. Improve Results (Tuning)
1. Algorithm Tuning: discovering the best models in model parameter space. This may include hyper parameter optimizations with additional helper services.
2. Ensemble Methods: where the predictions made by multiple models are combined.
3. Feature Engineering: where the attribute decomposition and aggregation seen in data preparation is tested further.


In [ ]:
# Using several techniques and testing the effectiveness on the data provided. 
# DecisionTreeRegressor depth 15 seems to be the of what we have tried
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge, Lasso, MultiTaskLasso, ElasticNet, BayesianRidge

seed = 7
scoring = 'accuracy'
models = []
models.append(('Lasso', Lasso()))
models.append(('Ridge', Ridge(alpha=0.05,normalize=True)))
models.append(('DecisionTreeRegressor', DecisionTreeRegressor(max_depth=4)))
models.append(('ElasticNet', ElasticNet()))
models.append(('BayesianRidge', BayesianRidge()))

minrms = rmsbase
for name, model in models:
    model.fit(train[predictors],train[target])
    predict = model.predict(test[predictors])
    rms = sqrt(mean_squared_error(test[target],predict))
    print(name, rms)
    if(minrms > rms):
        str = "Best Algorithm is: {}".format(name)
        minrms = rms
        
print("\n\n{}".format(str))


### Your Improve Results (If not happy with above situation)

## 5. Present Results
1. Context (Why): how the problem definition arose in the first place.
2. Problem (Question): describe the problem as a question.
3. Solution (Answer): describe the answer the the question in the previous step.
4. Findings: Bulleted lists of discoveries you made along the way that interests the audience. May include discoveries in the data, methods that did or did not work or the model performance benefits you observed.
5. Limitations: describe where the model does not work.
6. Conclusions (Why+Question+Answer)

### Results
1. Problem is wanting to make sure we can supply consumers with what they want when they want it
2. How can I make sure I have enough items but not too much of what a customer needs
3. By knowing what my sales are going to be I can plan accordingly
4. There are a broad set of items in the stores but they can be grouped into 3 categories. Stores of different types have different sizes. I know the list of items that are the big sellers across the board and those that are for a particular store. I can start to build some metrics that make up typical stores of a certain size or type.
5. Limited to the data being used. As more data is acquired more knowledge will be gained

### Present Your Results